In [2]:
import pandas as pd
import json
import os
import sys
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split  
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
from pandas.io.json import json_normalize
from sklearn.linear_model import LogisticRegression
from __future__ import absolute_import, print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
import re
import tweepy
import csv
import json
import time

C:\Users\mail2\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
consumer_key = 'Insert Your Key here'
consumer_secret = 'Insert Your Key here'
access_key = 'Insert Your Key here'
access_secret = 'Insert Your Key here'

In [3]:
df=pd.DataFrame(columns=('text', 'lat', 'lon','label'))

In [ ]:
class listener(StreamListener):
    def __init__(self, api=None):
        super(listener, self).__init__()
        self.num_tweets = 0
    
    def on_data(self, data):
        #See Tweepy documentation to learn how to access other fields
        
        try:
            js =(json.loads(data))
            tweet = (js["text"])
            print(tweet)
            
            la=[]
            lo=[]
            la.append(js['place']['bounding_box']['coordinates'][0][0][1])
            lo.append(js['place']['bounding_box']['coordinates'][0][0][0])
            la.append(js['place']['bounding_box']['coordinates'][0][1][1])
            lo.append(js['place']['bounding_box']['coordinates'][0][1][0])
            la.append(js['place']['bounding_box']['coordinates'][0][2][1])
            lo.append(js['place']['bounding_box']['coordinates'][0][2][0])
            la.append(js['place']['bounding_box']['coordinates'][0][3][1])
            lo.append(js['place']['bounding_box']['coordinates'][0][3][0])
            lats=sum(la)/4
            lons=sum(lo)/4
            #dat=(maxla/4,maxlo/4)
            #latlons.append(dat)
            #lats,lons = zip(*[dat])
            #print("lats",lats," long" ,lons)
            #df.insert({'text' : tweet , 'lat' : lats,'lon':lons} )
            df.loc[self.num_tweets] = [tweet, lats, lons,2]  # adding a row
            self.num_tweets += 1
            if self.num_tweets < 1000:
                return True
            else:
                return False

            
            
        except BaseException as e:
            print(status)
            

    def on_error(self, status):
        print(status)    
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
twitterStream = Stream(auth, listener())
twitterStream.filter(locations = [-10.97708574059, 50.6693012559, -5.03298539878, 58.1316222195],languages=["en"])

In [5]:
import pandas as pd
import json
import os
import sys
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split  
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
from pandas.io.json import json_normalize
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [6]:
X = df["text"]  
y = df["label"]
processed_tweets = []

for tweet in range(0, len(X)):  
    processed_tweet = re.sub(r"(?:\@|https?\://)\S+", "", str(X[tweet]))
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', processed_tweet)

    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)

    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 

    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
    
    #Removing punctuation
    processed_tweet = processed_tweet.translate(str.maketrans('', '', string.punctuation))

    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()

    processed_tweets.append(processed_tweet)


In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
X = loaded_vec.fit_transform(processed_tweets).toarray()

In [9]:
import pickle
pickle_in = open("ensemble model.sav","rb")
loaded_model = pickle.load(pickle_in)

In [11]:
df['label'] = loaded_model.predict(X)

C:\Users\mail2\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\mail2\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
df.shape

(1000, 5)

In [13]:
df1=df[df['label'] == '0' ].index

In [14]:
df1.shape
df.drop(df1 , inplace=True)

In [16]:
import folium
import pandas as pd
m = folium.Map(location=[53.384811, -6.263190],
                        zoom_start=8,
                        tiles="OpenStreetMap")
marker = folium.CircleMarker(location=[53.384811, -6.263190])
marker.add_to(m) 
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['lat'], df.iloc[i]['lon']], popup=df.iloc[i]['text']).add_to(m)
 
# Save it as html
m.save('bully_markers_on_folium_map1.html')